In [20]:
import numpy as np
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playerestimatedmetrics, commonplayerinfo
import unicodedata
import random
from gekko import GEKKO
from retry import retry

In [ ]:
#Params
k = 13.91

**utils**

In [11]:
def select_n_from_k(k, n):
  k_copy = list(k)
  random.shuffle(k_copy)
  return k_copy[:n]

**Get player years of service**

**Calculate player costs as max of their % salary cap and comparable player % salary cap**

In [12]:
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

In [13]:
def get_all_active_players():
  active_players = pd.json_normalize(players.get_active_players())
  return active_players

In [14]:
def get_epm_ranks(fname="epm_data.csv"):
  epm_data = pd.read_csv(fname)
  epm_data['epm_rank'] = epm_data['epm'].rank(method='min', ascending=False).astype('int')
  return epm_data[["nba_id", "name", "epm_rank"]]

In [41]:
def get_pct_salary_cap(fname="salary_data.csv", season="2023-24", salary_cap=136000000):
  salary_data = pd.read_csv(fname)
  salary_data[season] = salary_data[season].str[1:]
  salary_data[season] = salary_data[season].str.replace(',', '', regex=False)
  salary_data = salary_data[salary_data[season].notna()]
  salary_data[season] = salary_data[season].astype(int)
  salary_data["pct_salary_cap"] = salary_data[season] / salary_cap

  return salary_data[["Player", "pct_salary_cap"]]

In [42]:
def get_player_comparable_salary_cap():

  epm_ranks = get_epm_ranks()
  pct_salary_cap_data = get_pct_salary_cap()

  ranks = []

  for index, row in pct_salary_cap_data.iterrows():
    try:
      player_name = strip_accents(row["Player"])
      epm_rank = epm_ranks[epm_ranks["name"] == player_name]["epm_rank"].iloc[0]
      ranks.append(epm_rank)
    except:
      ranks.append(np.nan)

  pct_salary_cap_data["epm_rank"] = ranks
  pct_salary_cap_data = pct_salary_cap_data.dropna()

  pct_salary_cap_data_sorted = pct_salary_cap_data.sort_values(by='epm_rank').reset_index(drop=True)

  pct_salary_cap_data_sorted['AvgSalaryCapSurroundingCorrected'] = np.nan

  for i in range(len(pct_salary_cap_data_sorted)):
      surrounding_players = pct_salary_cap_data_sorted['pct_salary_cap'].iloc[max(i-3, 0):min(i+4, len(pct_salary_cap_data_sorted))].tolist()
      if surrounding_players:
          pct_salary_cap_data_sorted.at[i, 'AvgSalaryCapSurroundingCorrected'] = sum(surrounding_players) / len(surrounding_players)

  return pct_salary_cap_data_sorted

In [99]:
def get_player_costs():
  pct_salary = get_player_comparable_salary_cap()
  pct_salary['c'] = np.maximum(pct_salary['AvgSalaryCapSurroundingCorrected'], pct_salary['pct_salary_cap'])
  pct_salary["s"] = pct_salary["pct_salary_cap"]

  return pct_salary[["Player", "s", "c"]]

In [44]:
@retry()
def get_estimated_off_rating(player_names, season="2023-24"):
  estimated_metrics = playerestimatedmetrics.PlayerEstimatedMetrics(season=season, league_id="00", season_type="Regular Season")
  estimated_metrics = estimated_metrics.get_data_frames()[0]

  eo = []
  ed = []
  for player in player_names:
    cleaned_name = strip_accents(player)
    e_off_rtg = estimated_metrics[estimated_metrics["PLAYER_NAME"] == cleaned_name]["E_OFF_RATING"].iloc[0]
    e_def_rtg = estimated_metrics[estimated_metrics["PLAYER_NAME"] == cleaned_name]["E_DEF_RATING"].iloc[0]
    eo.append(float(e_off_rtg) / 100.0)
    ed.append(float(e_def_rtg) / 100.0)

  out = {
      "Player": player_names,
      "eo": eo,
      "ed": ed
  }

  return pd.DataFrame(out)

In [45]:
@retry()
def add_eo_ed(player_costs_df, season="2023-24"):
  e_mets_df = get_estimated_off_rating(player_costs["Player"].tolist(), season=season)
  player_costs_df = player_costs_df.merge(e_mets_df, on="Player")
  return player_costs_df

**Next, calculate the number of possessions per game for players**

In [46]:
def get_poss_per_game(fname="usage_data.csv"):
  usage_data = pd.read_csv(fname)
  usage_data["poss_per_game"] = usage_data["POSS"] / usage_data["GP"]
  return usage_data[["PLAYER", "poss_per_game"]]

In [47]:
def add_np(player_df):
  poss_per_game = get_poss_per_game()

  np = []
  for index, row in player_df.iterrows():
    player_name = strip_accents(row["Player"])
    poss_per_game_val = poss_per_game[poss_per_game["PLAYER"] == player_name]["poss_per_game"].iloc[0]
    np.append(poss_per_game_val)

  player_df["np"] = np
  return player_df

**Add years of service to player dataframe**

In [48]:
yos = pd.read_csv("PLAYERS_YOS.csv")

In [49]:
yos

,Unnamed: 0,id,full_name,YOS
0,0,1630173,Precious Achiuwa,3
1,1,203500,Steven Adams,10
2,2,1628389,Bam Adebayo,6
3,3,1630534,Ochai Agbaji,1
4,4,1630583,Santi Aldama,2
...,...,...,...,...
611,611,201152,Thaddeus Young,16
612,612,1629027,Trae Young,5
613,613,1630209,Omer Yurtseven,2
614,614,203469,Cody Zeller,10


In [50]:
def add_yos(player_df):
  yos_data = pd.read_csv("PLAYERS_YOS.csv")
  yos = []
  for index, row in player_df.iterrows():
    player_name = strip_accents(row["Player"])
    matching = yos_data[yos_data["full_name"] == player_name]
    if not matching.empty:
      yos.append(matching["YOS"].iloc[0])
    else:
      yos.append(np.nan)

  player_df["yos"] = yos
  return player_df

In [51]:
min_salary_mapping = {
    0: 953000,
    1: 1530000,
    2: 1710000,
    3: 1780000,
    4: 1840000,
    5: 1990000,
    6: 2150000,
    7: 2300000,
    8: 2460000,
    9: 2470000,
    10: 2720000
}

In [52]:
def add_min_eligible(player_df, min_salary_mapping, min_salary_prop_max):
  min_eligible = []
  for index, row in player_df.iterrows():
    if row["yos"] >= 10:
      yos = 10
    else:
      yos = row["yos"]

    if row["c"]* 136000000 <= min_salary_prop_max * min_salary_mapping[yos]:
      min_eligible.append(1)
    else:
      min_eligible.append(0)

  player_df["min_eligible"] = min_eligible
  return player_df


**Finally, calculate league-wide team average possessions per game**

In [32]:
def calculate_league_poss_per_game(fname="team_possession_data.csv"):
  team_possession_data = pd.read_csv(fname).dropna()
  team_possession_data["poss_per_game"] = team_possession_data["POSS"].str.replace(",","", regex=False).astype(int) / team_possession_data["GP"]
  return np.mean(team_possession_data["poss_per_game"])

In [100]:
player_costs = get_player_costs()
player_df = add_eo_ed(player_costs)
player_df = add_np(player_df)
player_df = add_yos(player_df)
player_df = player_df.drop_duplicates('Player', keep="last")
player_df = add_min_eligible(player_df.dropna(), min_salary_mapping, 1.5).drop_duplicates()
team_np = calculate_league_poss_per_game()

<ipython-input-52-047cfcab4938>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_df["min_eligible"] = min_eligible


In [101]:
player_df

,Player,s,c,eo,ed,np,yos,min_eligible
0,Joel Embiid,0.350054,0.350054,1.188,1.092,71.441176,7.0,0
1,Shai Gilgeous-Alexander,0.245492,0.315156,1.205,1.093,73.466667,5.0,0
2,Giannis Antetokounmpo,0.335589,0.335589,1.194,1.125,74.745763,10.0,0
3,Luka Dončić,0.294590,0.294590,1.184,1.143,79.314815,5.0,0
4,Nikola Jokić,0.350054,0.350054,1.202,1.103,70.066667,8.0,0
...,...,...,...,...,...,...,...,...
452,Jalen McDaniels,0.033206,0.033206,1.032,1.109,18.228571,4.0,0
453,AJ Griffin,0.027301,0.027301,1.115,1.200,16.555556,1.0,0
454,Johnny Davis,0.037138,0.037138,1.024,1.100,21.666667,1.0,0
456,Maxwell Lewis,0.008232,0.022938,0.785,1.119,7.857143,0.0,0


In [103]:
player_df.to_csv("player_df.csv")

In [104]:
player_df[player_df["min_eligible"] == 1]

,Player,s,c,eo,ed,np,yos,min_eligible
289,Derrick Rose,0.023503,0.028619,1.036,1.063,35.166667,14.0,1
311,Alex Len,0.014851,0.023568,1.185,1.020,18.615385,10.0,1
312,John Konchar,0.017647,0.017647,1.037,1.099,40.765957,4.0,1
356,Justin Holiday,0.014851,0.019674,1.141,1.121,30.131579,10.0,1
358,Josh Richardson,0.021261,0.025592,1.084,1.124,52.720930,8.0,1
370,Svi Mykhailiuk,0.014851,0.021259,1.131,1.063,16.576923,5.0,1
392,Mike Muscala,0.025735,0.028935,1.086,1.157,29.342105,10.0,1
393,Garrett Temple,0.014851,0.028838,1.044,1.157,9.625000,13.0,1
396,Jericho Sims,0.014176,0.018242,1.076,1.112,25.857143,2.0,1
398,Kessler Edwards,0.014176,0.014832,1.041,1.061,10.384615,2.0,1


In [ ]:
team_np

100.14406309195043

In [ ]:
player_df

,Player,c,eo,ed,np
0,Joel Embiid,0.387052,1.188,1.092,71.441176
1,Shai Gilgeous-Alexander,0.348465,1.204,1.093,73.466667
2,Giannis Antetokounmpo,0.371058,1.195,1.121,74.745763
3,Luka Dončić,0.325725,1.182,1.152,79.314815
4,Nikola Jokić,0.387052,1.201,1.104,70.066667
...,...,...,...,...,...
453,AJ Griffin,0.030186,1.115,1.200,16.555556
454,Johnny Davis,0.041063,1.047,1.109,21.666667
455,Ish Smith,0.027255,1.018,1.186,36.023256
456,Maxwell Lewis,0.025363,0.792,1.129,7.857143


**Brute force won't work - way too many combos**

**Finally, we are ready to set up the constrained optimization problem.**

In [122]:
def objective_function(x, player_df):
  pts_added = 0
  pts_given = 0

  for i in range(n):
    player_data = player_df.iloc[i]

    term = float(player_data["eo"]) * float(player_data["np"]) * float(x[i])
    pts_added += term

    term = float(player_data["ed"]) * float(player_data["np"]) * float(x[i])
    pts_given += term

  pts_added = pts_added ** 13.91
  pts_given = pts_given ** 13.91

  return (pts_added) / (pts_added + pts_given)

**Model 3: with minimum salary exception**

In [156]:
player_df = pd.read_csv("player_df.csv")
player_df = player_df.dropna()
player_list = player_df["Player"].tolist()
fixed_players = [
    "Donovan Mitchell",
    "Darius Garland",
    "Evan Mobley",
    "Jarrett Allen"
]
available_vets = [
    "Caris LeVert",
    "Tristan Thompson"
]
num_years_on_team = [
    3,
    1,
]
available_players = [i for i in pd.read_csv("FreeAgents.csv")["Player"].tolist() if i in player_list]
refined_player_df = player_df[player_df["Player"].isin(available_players) | player_df["Player"].isin(fixed_players) | player_df["Player"].isin(available_vets)]
refined_player_list = refined_player_df["Player"].tolist()
fixed_players_indices = [refined_player_list.index(i) for i in fixed_players]
available_vets_indices = [refined_player_list.index(i) for i in available_vets]

In [157]:
def bird_eligible_array(player_df, fixed_players_indices, num_years_on_team):
  ind = 0
  bird_eligible = []
  early_bird = []
  non_bird = []
  for index, row in player_df.iterrows():
    if ind in fixed_players_indices:
      num_years = num_years_on_team[fixed_players_indices.index(ind)]
      if num_years >= 3:
        bird_eligible.append(1)
        early_bird.append(1)
        non_bird.append(1)
      elif num_years == 2:
        bird_eligible.append(0)
        early_bird.append(1)
        non_bird.append(1)
      elif num_years == 1:
        bird_eligible.append(0)
        early_bird.append(0)
        non_bird.append(1)
      else:
        bird_eligible.append(0)
        early_bird.append(0)
        non_bird.append(0)

    else:
      bird_eligible.append(0)
      early_bird.append(0)
      non_bird.append(0)
    ind += 1
  return bird_eligible, early_bird, non_bird


In [158]:
refined_player_df["b"], refined_player_df["eb"], refined_player_df["nb"] = bird_eligible_array(refined_player_df, available_vets_indices, num_years_on_team)

<ipython-input-158-a7edcc95a24d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refined_player_df["b"], refined_player_df["eb"], refined_player_df["nb"] = bird_eligible_array(refined_player_df, available_vets_indices, num_years_on_team)
<ipython-input-158-a7edcc95a24d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  refined_player_df["b"], refined_player_df["eb"], refined_player_df["nb"] = bird_eligible_array(refined_player_df, available_vets_indices, num_years_on_team)
<ipython-input-158-a7edcc95a24d>:1

In [160]:
refined_player_df[refined_player_df["nb"] == 1]

,Unnamed: 0,Player,s,c,eo,ed,np,yos,min_eligible,b,eb,nb
110,114,Caris LeVert,0.113122,0.130807,1.147,1.095,58.122449,7.0,0,1,1,1
152,156,Tristan Thompson,0.014851,0.037471,1.184,1.122,25.277778,12.0,0,0,0,1


In [161]:
m = GEKKO(remote=False)
n = len(refined_player_df)
salaries = refined_player_df["c"].tolist()
actual_salaries = refined_player_df["s"].tolist()
min_eligible = refined_player_df["min_eligible"].tolist()
poss = refined_player_df["np"].tolist()

#x = [m.Var(lb=0, ub=1, integer=True) for _ in range(n)]
#x = [m.Var(value=1 if i < 15 else 0, lb=0, ub=1, integer=True) for i in range(n)]
x = [m.Var(value=1 if i in fixed_players_indices else 0, lb=0, ub=1, integer=True) for i in range(n)]
print(x)

pts_added = m.Intermediate(1)
pts_given = m.Intermediate(1)

for i in range(n):
    player_data = player_df.iloc[i]
    eo = player_data["eo"]
    np = player_data["np"]
    ed = player_data["ed"]

    #Intermediate terms for readability
    term_added = m.Intermediate(eo * np * x[i])
    term_given = m.Intermediate(ed * np * x[i])

    #Update pts_added and pts_given
    pts_added = m.Intermediate(pts_added + term_added)
    pts_given = m.Intermediate(pts_given + term_given)

pts_added_powered = m.Intermediate(pts_added**13.91)
pts_given_powered = m.Intermediate(pts_given**13.91)

objective = pts_added_powered / (pts_added_powered + pts_given_powered)

m.Maximize(objective)

#Add roster size constraint
m.Equation(m.sum(x) <= 15)
m.Equation(m.sum(x) >= 12)

#Add salary cap constraint
salaries_var = [m.Const(value=salaries[i]) for i in range(n)]
actual_salaries_var = [m.Const(value=actual_salaries[i]) for i in range(n)]

all_salaries = [salaries_var[i] * x[i] for i in range(n) if (i not in fixed_players_indices)]
all_salaries = all_salaries + [actual_salaries_var[i] for i in fixed_players_indices]
m.Equation(m.sum(all_salaries) <= 1.5)
m.Equation(m.sum([salaries_var[i] * x[i] * (1 - min_eligible[i]) for i in range(n) if i not in fixed_players_indices]) <= 1)

#Add early bird salary constraint - require that, if player is included with bird rights, then cost <= 1.75 * last salary
b_var = [m.Const(value=refined_player_df.iloc[i]["b"]) for i in range(n)]
eb_var = [m.Const(value=refined_player_df.iloc[i]["eb"]) for i in range(n)]

for i in range(n):
    condition = m.Intermediate(x[i] * (1 - b_var[i]) * eb_var[i])
    #This is a trick to apply conditional constraints: we use m.max3 to ensure c[i] <= 1.75 * s[i] when condition is 1
    #and ignore the constraint when condition is 0 by setting a high upper limit.
    m.Equation(salaries_var[i] <= m.if3(condition, 1.75 * actual_salaries[i], 1e5))

#Add minimum salary constraint
m.Equation(m.sum([salaries_var[i] * x[i] for i in range(n)]) >= .9)

#Add play time constraint
#poss_var = [m.Const(value=poss[i]) for i in range(n)]
#m.Equation(m.sum([poss_var[i] * x[i] for i in range(n)]) >= 5*team_np)

#Add fixed players constraint
for idx in fixed_players_indices:
    m.Equation(x[idx] == 1)

m.options.SOLVER=1
m.solve(disp=True)
solution = [x[i].value[0] for i in range(n)]




[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            5
   Constants    :          524
   Variables    :         1185
   Intermediates:          659
   Connections  :          656
   Equations    :         1582
   Residuals    :          923
 
 Number of state variables:           1185
 Number of total equations: -          927
 Number of slack variables: -          398
 -

In [162]:
for i in range(len(solution)):
  if solution[i] == 1.0:
    print(refined_player_df.iloc[i])

Unnamed: 0                     5
Player          Donovan Mitchell
s                       0.243838
c                       0.278918
eo                         1.156
ed                         1.059
np                     74.617021
yos                          6.0
min_eligible                   0
b                              0
eb                             0
nb                             0
Name: 5, dtype: object
Unnamed: 0                 8
Player          LeBron James
s                   0.350054
c                   0.350054
eo                     1.148
ed                     1.127
np                      74.0
yos                     20.0
min_eligible               0
b                          0
eb                         0
nb                         0
Name: 8, dtype: object
Unnamed: 0               51
Player          Evan Mobley
s                  0.065314
c                  0.089307
eo                    1.098
ed                     1.07
np                65.421053
yos           

In [163]:
objective_function(solution, refined_player_df)

0.5317429641489484